# Adding layer number to streamlines output

**A method to add information about streamlines to output shape-file**

This script uses a shape-file containing the points of streamlines exported from a FEFLOW simulation and adds a column with the according layer number of each point.

Firstly the files have to be defined: a fem-file is needed as reference to locate the exported points, shp is the exported shapefile and shp2 the edited shapefile that will be saved in the end.

In [1]:
import sys
import geopandas as gpd
import ifm_contrib as ifm

print("FEFLOW version: ", ifm.getKernelVersion())

fem = "../../data/unittests/models/box_mlw.fem"
shp = "../../data/unittests/import+export/streamlines.shp"
shp2 = "../../data/unittests/import+export/streamlines_ln.shp"

FEFLOW version:  7305


This function will detect the layer number of each point of the exported shp. 

*Note that due to numerical roundings points might lay slightly outside the model. In this case "none" is applied instead of layer number.*

In [2]:
def findLayer(doc, x, y, z):
    element = doc.findElementAtXYZ(x, y, z)
    layer = 0
    for l in range(1,lll+1):
        
        if type(element)!= int:
            layer = "none"
            break
            
        elif element - l*ee <= 0:
            layer = l
            break
        
        else:
            continue
        
    return(layer)

In [3]:
gdf = gpd.read_file(shp)

doc = ifm.loadDocument(fem)

origin_x = doc.getOriginX()
origin_y = doc.getOriginY()
origin_z = doc.getOriginZ()

gdf ['x'] = gdf['X'] - origin_x
gdf ['y'] = gdf['Y'] - origin_y
gdf ['z'] = gdf['Z'] - origin_z

ee = doc.getNumberOfElementsPerLayer()
lll = doc.getNumberOfLayers()

In [4]:
gdf['layer_nr'] = gdf.apply(lambda row: findLayer(doc, row['x'], row['y'], row['z']), axis=1)
gdf.to_file(shp2)